In [1]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_excel('novos dados.xlsx')

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [4]:
df_limpo = pd.DataFrame(index = df.index)
df_limpo['title'] = df['title']

In [5]:
mask = ['classification','reviews']
treinar = df[mask]
treinar

,classification,reviews
0,4.1,32
1,4.4,3
2,4.2,18
3,4.7,17
4,4.3,108
...,...,...
669,3.1,12
670,3.0,1
671,3.0,2
672,2.9,5


In [6]:
y = df['target']

## 3. Features

In [7]:
# mask_train = df_limpo['date'] < "2019-04-01"
# mask_val = (df_limpo['date'] >= "2019-04-01")

title_train = df_limpo[:300]['title']
title_val = df_limpo[300:]['title']

# Xtrain, Xval = features[mask_train], features[mask_val]
# ytrain, yval = y[mask_train], y[mask_val]
Xtrain, Xval = treinar[:300], treinar[300:]
ytrain, yval = y[:300], y[300:]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((300, 2), (374, 2), (300,), (374,))

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

# title_train = df_limpo[mask_train]['title']
# title_val = df_limpo[mask_val]['title']
title_train = df_limpo[:300]['title']
title_val = df_limpo[300:]['title']

title_vec = TfidfVectorizer(min_df=4, ngram_range=(1,5))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)


In [28]:
title_bow_train.shape

(300, 435)

In [29]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [30]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((300, 437), (374, 437))

# 4 Random Forest

In [31]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf=1, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=6,
                       random_state=0)

In [32]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [33]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [34]:
average_precision_score(yval, p)

0.30155719798566594

In [35]:
roc_auc_score(yval, p)

0.4908476485497043

# 5 LightGBM

In [17]:
from lightgbm import LGBMClassifier

In [36]:
mdl = LGBMClassifier(learning_rate= 0.012750003188347398, num_leaves= 16, max_depth=6, 
                         min_child_samples=7, subsample=0.6283707044270589,
                         colsample_bytree=0.19713289098659126, bagging_freq=1,n_estimators=685, random_state=0, 
                         class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

LGBMClassifier(bagging_freq=1, class_weight='balanced',
               colsample_bytree=0.19713289098659126,
               learning_rate=0.012750003188347398, max_depth=6,
               min_child_samples=7, n_estimators=685, n_jobs=6, num_leaves=16,
               random_state=0, subsample=0.6283707044270589)

In [37]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

C:\Users\Gustavo\Anaconda3\envs\to_deploy\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [26]:
space = [(1e-3, 1e-1, 'log-uniform'), # lr
          (1, 15), # max_depth
          (1, 20), # min_child_samples
          (0.05, 1.), # subsample
          (0.05, 1.), # colsample_bytree
          (100,1000), # n_estimators
          (1,5), # min_df
          (1,5)] # ngram_range
res.x

[0.012750003188347398,
 6,
 7,
 0.6283707044270589,
 0.19713289098659126,
 685,
 4,
 5]

In [ ]:
[0.012250750764764855, 8, 6, 0.5976582413192033, 0.2474882432951916, 516, 4, 4]

In [38]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.3474028562435173, 0.48285694170656157)

In [ ]:
(0.27455058995659876, 0.4762390875809631)

# 6 Bayesian Optimization

In [21]:
from skopt import forest_minimize

In [25]:
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    min_df = params[6]
    ngram_range = (1, params[7])
    
    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    title_bow_train = title_vec.fit_transform(title_train)
    title_bow_val = title_vec.transform(title_val)
    
    Xtrain_wtitle = hstack([Xtrain, title_bow_train])
    Xval_wtitle = hstack([Xval, title_bow_val])
    
    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                         class_weight="balanced", n_jobs=6)
    mdl.fit(Xtrain_wtitle, ytrain)
    
    p = mdl.predict_proba(Xval_wtitle)[:, 1]
    
    print(roc_auc_score(yval, p))
    
    return -average_precision_score(yval, p)


space = [(1e-3, 1e-1, 'log-uniform'), # lr
          (1, 15), # max_depth
          (1, 20), # min_child_samples
          (0.05, 1.), # subsample
          (0.05, 1.), # colsample_bytree
          (100,1000), # n_estimators
          (1,5), # min_df
          (1,5)] # ngram_range

res = forest_minimize(tune_lgbm, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272, 3, 1]


C:\Users\Gustavo\Anaconda3\envs\to_deploy\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')
C:\Users\Gustavo\Anaconda3\envs\to_deploy\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.4761686848774993
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.2922
Function value obtained: -0.3102
Current minimum: -0.3102
Iteration No: 2 started. Evaluating function at random point.
[0.053887464791860025, 1, 15, 0.7437489153990157, 0.8675167974293533, 549, 3, 4]
0.4841241903689102
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.1895
Function value obtained: -0.3010
Current minimum: -0.3102
Iteration No: 3 started. Evaluating function at random point.
[0.004151454520895999, 6, 20, 0.8682075103820793, 0.9491436163200662, 411, 4, 3]


C:\Users\Gustavo\Anaconda3\envs\to_deploy\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.48496902281047605
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.2264
Function value obtained: -0.2806
Current minimum: -0.3102
Iteration No: 4 started. Evaluating function at random point.
[0.0014099928811969545, 9, 9, 0.6502182010234373, 0.6866210554187129, 828, 5, 2]


...

0.47990002816108135
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 0.5755
Function value obtained: -0.3345
Current minimum: -0.3453
Iteration No: 49 started. Searching for the next optimal point.
[0.005919393762796261, 11, 7, 0.6404927945042223, 0.11618422994721038, 719, 4, 4]


C:\Users\Gustavo\Anaconda3\envs\to_deploy\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.4961278513094903
Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 1.0781
Function value obtained: -0.3213
Current minimum: -0.3453
Iteration No: 50 started. Searching for the next optimal point.
[0.0034175211209914918, 6, 4, 0.6357006149183271, 0.1453736165293555, 984, 4, 5]


C:\Users\Gustavo\Anaconda3\envs\to_deploy\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.4712756969867643
Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 1.0290
Function value obtained: -0.3308
Current minimum: -0.3453


# 7 Logistic Reg

In [39]:
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix

In [129]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

scaler = StandardScaler()
# scaler = MaxAbsScaler()


Xtrain_wtitle2[:, :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())

# Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
# Xval_wtitle2 = scaler.transform(Xval_wtitle2)

C:\Users\Gustavo\Anaconda3\envs\to_deploy\lib\site-packages\scipy\sparse\_index.py:124: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [130]:
Xval_wtitle2.shape

(374, 437)

In [131]:

mdl = LogisticRegression(C=0.0001,n_jobs=6, random_state=0)
mdl.fit(Xtrain_wtitle2, ytrain)

LogisticRegression(C=0.0001, n_jobs=6, random_state=0)

In [132]:
p = mdl.predict_proba(Xval_wtitle2)[:, 1]

In [133]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.33792352917966934, 0.5337580963108984)

In [ ]:
(0.3238428572594492, 0.4953886229231203) - sem tuning, standardscaler

(0.2933647722524177, 0.46627710504083353) - sem tuning, maxabsscaler
(0.30539232757962287, 0.4772599267811884) - C=10, standardscaler

(0.27113551861312046, 0.4392424669107293) - C=10, maxabsscaler
(0.33792352917966934, 0.5337580963108984) - C=0.0001, standardscaler